In [ ]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

from pathlib import Path
import pandas as pd

plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

In [ ]:
INF = 999999

def plot_explored(f, steps=-1, box=INF, l=6):
    t = pd.read_csv(f, delimiter=',', dtype=None)
    total_t = len(t.index)
 
    max_steps = len(t) if steps == -1 else steps
    
    if box == INF:
        n, m = max(t[:,0]) + 1, max(t[:,1]) + 1
    else:
        n, m = box, box
    
    img = np.ones((n, m, 4))
    img.fill(0.97)  # grey
    
    subt = t.loc[(t.i < box) & (t.j < box)]
    
    for index, row in subt.loc[subt['type'] == 'Expanded'].iterrows():
        assert row.step != -1
        if row.step > max_steps:
            continue
        val = row.step / max_steps
        if val <= 1.0:
            img[row.i, row.j] = cm.cool(val)
        
    for index, row in subt.loc[subt['type'] == 'Path'].iterrows():
        img[row.i, row.j] = mcolors.to_rgba('xkcd:black')
        
    for index, row in subt.loc[subt['type'] == 'Match'].iterrows():
        for d in range(l):
            if row.i+d < img.shape[0] and row.j+d < img.shape[1]:
                img[row.i+d, row.j+d] = mcolors.to_rgba('xkcd:canary')
    
    fig, ax = plt.subplots(figsize=(20,16))
    title = f.stem
    ax.set_title('{} ({})'.format(title, total_t), y=-0.01)
    ax.xaxis.tick_top()
    ax.grid(False)
    
    ax.imshow(img)
    fig.savefig('imgs/'+title+'.png')

In [ ]:
d = Path('stats/')
min_steps = 10000000
all_files = list(d.glob('*.csv'))
some_files = [d/'exact.csv', d/'inexact.csv', d/'exact_pruning.csv', d/'inexact_pruning.csv']

files = some_files
for f in files:
    t = pd.read_csv(f, delimiter=',', dtype=None)
    #display(t)
    min_steps = min(min_steps, max(t['step']))
for f in files:
    plot_explored(f, box=350, steps=4000, l=4)

In [ ]:
def plot_l():
    f = 'stats/table.csv'
    t = pd.read_csv(f, delimiter=',', dtype=None)
    #display(t)
    t['algo'] = t.heuristic + ' with ' + t.distance_function
    t['band'] = t.explored / t.len_a
    subt = t.loc[(t.error_rate == 0.1)]
    #display(subt)

    fig, ax = plt.subplots()

    for key, grp in subt.groupby(['algo']):
        ax = grp.plot(ax=ax, kind='line', x='l', y='band', label=key)

    plt.legend(loc='best')
    plt.show()

plot_l()

In [ ]:
def plot_():
    f = 'stats/table.csv'
    t = pd.read_csv(f, delimiter=',', dtype=None)
    t['algo'] = t.heuristic + ' with ' + t.distance_function
    t['band'] = t.explored / t.len_a
    t['delta'] = t.distance - t.root_h
    
    subt = t.loc[(t.error_rate == 0.1)]
    #display(subt)

    fig, ax = plt.subplots()

    for key, grp in subt.groupby(['algo']):
        ax = grp.plot(ax=ax, kind='line', x='delta', y='band', label=key)

    plt.legend(loc='best')
    plt.show()

plot_()